# Interactive SEIRDS model

This notebook contains code for runnning an SEIRDS model for disease propagation.

We will use the Euler method to numerically sovle coupled first order ODE's for the SEIRDS ODE model.

S = Susceptible, E = Exposed, I = Infected, R = Recovered, D = Dead, N = Total = S+E+I+R+D

$\frac{dS}{dt}=-SI\frac{q\beta}{N}+\tau R$

$\frac{dE}{dt}=SI\frac{q\beta}{N}-\delta E$

$\frac{dI}{dt}=\delta E-(1-\alpha)\gamma I-\alpha\rho I$

$\frac{dR}{dt}=(1-\alpha)\gamma I-\tau R$

$\frac{dD}{dt}=\alpha\rho I$

Here, $\beta$ represents the average number of people that come within infection range of an infected individual per day, which is given by $\frac{1}{\text{number of individuals}}$. $\gamma$ is the remove probability per day, which is given by $\frac{1}{\text{recovery time}}$. $\delta$ is the rate at which exposed individuals become infected (incubation period), which is given by $\frac{1}{\text{incubation period}}$. $\alpha$ is the percentage of infected people who die. $\rho$ is the rate at which people die, given as $\frac{1}{\text{time infected before death}}$. q is the mitigation factor due to quarantining (between 0 and 1), where 0 is perfect quarantine and 1 is no quarantine. $\tau$ is the rate which recovered individuals return to the susceptible population due to loss of immunity.

Toggling the sliders will change the output.

Below we import packages:

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import math
import sys

Next we define a function that has the variables we want to interact with that will output the plot we want to generate. See SEIRDS model notebook for more details on how the function works.

In [7]:
@interact(dt=(0.0001,0.1,0.001), tottime=(100,750,5), beta=(0,1,0.01), gamma=(0,1,0.01), rho=(0,1,0.01), delta=(0,1,0.01),
          alpha=(0,1,0.01), tau=(0,1,0.0001), q=(0,1,0.01), N=(0,100000,1000), pr=(0,1,0.0001), pe=(0,1,0.0001), pi=(0,1,0.0001))
def SIR_model(dt=0.01, tottime=365, beta=0.25, gamma=0.07, rho=0.05, delta=0.2, alpha=0.05, tau=0.01, q=1, N=50000, pr=0.0, pe=0.0, pi=0.0001):
    #initializing parameters
    #initial percent susceptible
    ps = 1-pr-pe-pi
    
    niter = int(math.ceil(tottime/dt))
    t = np.arange(0, tottime, dt)   
    S = np.zeros(niter)
    E = np.zeros(niter)
    I = np.zeros(niter)
    R = np.zeros(niter)
    D = np.zeros(niter)
    
     
    S[0] = ps*N
    E[0] = pe*N
    I[0] = pi*N
    R[0] = pr*N
    D[0] = 0

    for j in range(niter-1):
        dSdt = -q*beta/N*S[j]*I[j] + tau*R[j]
        dEdt = q*beta/N*S[j]*I[j] - delta*E[j]
        dIdt = delta*E[j] - (1-alpha)*gamma*I[j] - alpha*rho*I[j]
        dRdt = (1-alpha)*gamma*I[j] - tau*R[j]
        dDdt = alpha*rho*I[j]
        S[j+1] = S[j] + dt*dSdt
        E[j+1] = E[j] + dt*dEdt
        I[j+1] = I[j] + dt*dIdt
        R[j+1] = R[j] + dt*dRdt
        D[j+1] = D[j] + dt*dDdt
    N = S + E + I + R + D
    
    plt.plot(t, S)
    plt.plot(t, E)
    plt.plot(t, I)
    plt.plot(t, R)
    plt.plot(t, D)
    plt.plot(t, N)
    plt.legend(['susceptible', 'exposed', 'infected', 'recovered', 'dead', 'total'], loc = 'upper right')
    plt.title('SEIRDS model')
    plt.xlabel('days elapsed since ' + str(pi*100) + ' percent of the population became infected')
    plt.ylabel('population')
    
    print('dt = ' + str(dt), 'tottime = ' + str(tottime), 'beta = ' + str(beta), 'gamma = ' + str(gamma),
      'rho = ' + str(rho), 'delta = ' + str(delta), 'alpha = ' + str(alpha), 'tau = ' + str(tau), 'q = ' + str(q),
          'N[0] = ' + str(N[0]), 'pr = ' +str(pr), 'pe = ' + str(pe), 'pi = ' + str(pi))
   
    return plt.show()

interactive(children=(FloatSlider(value=0.01, description='dt', max=0.1, min=0.0001, step=0.001), IntSlider(va…